# HMM的简单应用：中文分词

### 作业要求：
1. 阅读hmm.py的实现，用最大似然法估计HMM参数（见Q1）
2. （选做）优化基本模型（见Q2）

##### 提交时，单独提交本ipynb文件即可。若对hmm.py做了修改，请打包提交。


In [40]:
# -*-coding:utf-8
# By tostq <tostq216@163.com>
# 博客: blog.csdn.net/tostq
import numpy as np
import hmm
from sklearn.model_selection import train_test_split


# 获得某词的分词结果
# 如：（我：S）、（你好：BE）、（恭喜发财：BMME）
# 0 1 2 3 分别表示 B M E S
def getList(input_str):
    outpout_str = []
    if len(input_str) == 1:
        outpout_str.append(3)
    elif len(input_str) == 2:
        outpout_str = [0, 2]
    else:
        M_num = len(input_str) - 2
        M_list = [1] * M_num
        outpout_str.append(0)
        outpout_str.extend(M_list)
        outpout_str.append(2)
    return outpout_str


# 预处理词典：RenMinData.txt_utf8
def precess_data():
    ifp = open("RenMinData.txt", 'r').readlines()
    line_num = 0
    word_dic = {}
    word_ind = 0
    line_seq = []
    state_seq = []
    # 保存句子的字序列及每个字的状态序列，并完成字典统计
    for line in ifp:
        line_num += 1
        #print(line)
        line = line.strip()
        #print(line)
        word_list = []
        for i in range(len(line)):
            if line[i] == " ": continue
            word_list.append(line[i])
            # 建立单词表
            #print(word_list)
            if not (line[i]) in word_dic.keys():
                word_dic[line[i]] = word_ind
                word_ind += 1
        line_seq.append(word_list)
        
        lineArr = line.split(" ")
        #print(lineArr)
       
        line_state = []
        for item in lineArr:
            line_state += getList(item)
            #print(item)
            #print(line_state)
        state_seq.append(np.array(line_state))

    lines = []
    for i in range(line_num):
        lines.append(np.array([[word_dic[x]] for x in line_seq[i]]))

    return lines, state_seq, word_dic

In [41]:
X, Z, word_dic = precess_data() # 预处理数据集，得到观测值序列集合X，状态序列集合Z，词典word_dic
X_train, X_test, Z_train, Z_test = train_test_split(X, Z, test_size=0.1, random_state=100) # 划分出一部分测试集

In [74]:
print(Z[3][1])

0


# Q1. 请你使用最大似然估计法求解HMM的参数

In [83]:
# 用最大似然估计求解HMM的参数
def MLE(X, Z, num_words):
    # 你需要重写代码，正确地估计这三个矩阵的参数
    start_prob = np.zeros(4)
    transmat_prob = np.zeros((4,4))
    emission_prob = np.zeros((4, len(word_dic)))
    num0 = 0
    num3 = 0
    for i in Z:
        if i[0] == 0:
            num0 += 1
        else:
            num3 += 1
    start_prob[0] = num0 / len(Z)
    start_prob[1] = 0
    start_prob[2] = 0
    start_prob[3] = num3 / len(Z)
    #计算转移矩阵
    for i in range(4):
        for j in range(4):
            for k in Z:
                if len(k) == 1:
                    continue
                for l in range(len(k) - 1):
                    if k[l] == i and k[l+1] == j:
                        transmat_prob[i][j] += 1
    
                        
    for i in range(len(X)):
        for j in range(len(X[i])):
            emission_prob[Z[i][j]][X[i][j]] += 1
   
            
    # 对概率矩阵归一化
    start_prob = start_prob / np.sum(start_prob)
    transmat_prob = transmat_prob / np.repeat(np.sum(transmat_prob, axis=1), 4).reshape((4, 4))
    emission_prob = emission_prob / np.repeat(np.sum(emission_prob, axis=1), num_words).reshape((4, num_words))
    
    return start_prob, transmat_prob, emission_prob

In [84]:
start_prob, transmat_prob, emission_prob = MLE(X_train, Z_train, len(word_dic)) # 用最大似然估计求解参数
wordseg_hmm = hmm.DiscreteHMM(start_prob, transmat_prob, emission_prob, 4, len(word_dic))

print("startprob_prior: ", wordseg_hmm.start_prob)
print("transmit: ", wordseg_hmm.transmat_prob)


startprob_prior:  [0.58244426 0.         0.         0.41755574]
transmit:  [[0.         0.1168293  0.8831707  0.        ]
 [0.         0.27707947 0.72292053 0.        ]
 [0.46893362 0.         0.         0.53106638]
 [0.43019261 0.         0.         0.56980739]]
(4, 3728)
3728


# Q2（加分项）. 请你尝试优化这个基本模型，可以从时间上优化，也可以从最后的F1指标上优化。

## 下面是测试部分

如果正确地估计出了HMM的参数，得到的结果应该接近：
- Recall: 0.767
- Precise: 0.782
- F1 score: 0.774

In [85]:
# 测试
correct_words = 0 # 正确分出的词
out_num_words = 0 # 解码得到的词数
ans_num_words = 0 # 正确答案中的词数

for i in range(len(X_test)):
    z = wordseg_hmm.decode(X_test[i])
    y = (z == Z_test[i])
    last_end = -1
    for j in range(len(X_test[i])):
        if z[j] >= 2:   # 2 和 3 表示一个词的结尾
            out_num_words += 1
            correct_words += min(y[last_end + 1:j + 1])  # 这个词的每个位置的状态和正确答案全部相同
            last_end = j
        if Z_test[i][j] >= 2:
            ans_num_words += 1

recall = float(correct_words) / float(ans_num_words)   # 召回率 = 正确分出的词 / 正确答案中的次数
precise = float(correct_words) / float(out_num_words)  # 准确率 = 正确分出的词 / 解码得到的词数
F1_score = 2 * recall * precise / (recall + precise)

print("Recall: %6.3f\tPrecise: %6.3f\tF1 score: %6.3f" % (recall, precise, F1_score))

/Users/zhaohan/Desktop/python程序设计与数据挖掘导论/HW8-release/hmm.py:127: RuntimeWarning: invalid value encountered in true_divide
  alpha[0] = alpha[0] / c[0]
/Users/zhaohan/Desktop/python程序设计与数据挖掘导论/HW8-release/hmm.py:101: RuntimeWarning: divide by zero encountered in double_scalars
  max_pro_state[0] = self.emit_prob(X[0]) * self.start_prob * (1 / c[0])  # 初始概率
/Users/zhaohan/Desktop/python程序设计与数据挖掘导论/HW8-release/hmm.py:101: RuntimeWarning: invalid value encountered in multiply
  max_pro_state[0] = self.emit_prob(X[0]) * self.start_prob * (1 / c[0])  # 初始概率
/Users/zhaohan/Desktop/python程序设计与数据挖掘导论/HW8-release/hmm.py:108: RuntimeWarning: divide by zero encountered in double_scalars
  max_pro_state[i][k] = np.max(prob_state) * (1 / c[i])
/Users/zhaohan/Desktop/python程序设计与数据挖掘导论/HW8-release/hmm.py:108: RuntimeWarning: invalid value encountered in double_scalars
  max_pro_state[i][k] = np.max(prob_state) * (1 / c[i])


Recall:  0.767	Precise:  0.782	F1 score:  0.775
